In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# A little bit of environmental variable toggling to account for KMeans data leak nonsense
# Any warnings here are not ignored just so people are aware of the risks
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file

# Make the scaler
scaler = StandardScaler()

# Fit scaler to data
scaler = scaler.fit(df_market_data)

# Normalize and transform data
scaled_data = scaler.transform(df_market_data)

# Convert to DataFrame for easier viewing
scaled_df = pd.DataFrame(scaled_data, columns=df_market_data.columns)

# Retrieve the index from the original dataset
scaled_df = scaled_df.set_index(df_market_data.index)
scaled_df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351


In [6]:
scaled_df.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

It's not quite there with some of the more volatile cryptocurrencies fluctuating, but a difference of 6 is preferable to a difference of 8,000.

---

### Find the Best Value for k Using the Original Data.

In [7]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [8]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `scaled_df`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, n_init="auto")
    model.fit(scaled_df)
    inertia.append(model.inertia_)

C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known

In [9]:
# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame({"k":k, "inertia":inertia})
elbow_df

,k,inertia
0,1,287.000000
1,2,198.571818
2,3,123.190482
3,4,79.022435
4,5,65.697046
5,6,55.559954
6,7,43.219959
7,8,40.667644
8,9,33.306690
9,10,28.794016


In [10]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(x="k", y="inertia", xticks=k, title="Cryptocurrency Clustering Elbow Plot")

:Curve   [k]   (inertia)

The best value for `k` here seems to be **4** as any decreases in inertia after that point becomes marginal and is not worth the increase of cluster complexity or computing time. This is denoted by how an "elbow" forms at `k=4` on the graph and the slant of the slope flattens out drastically after that point.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [11]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

# Fit the K-Means model using the scaled data
model.fit(scaled_df)

# Predict the clusters to group the cryptocurrencies using the scaled data
predictions = model.predict(scaled_df)

C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [12]:
# Print the resulting array of cluster values.
# You'll need a separate module like k-means-constrained if you want to control the minimum cluster size for this model
predictions

array([0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3, 0,
       3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 2, 0, 3, 3, 1, 3, 3, 3, 3])

In [13]:
# Create a copy of the original, unscaled DataFrame
cluster_df = df_market_data.copy()

In [14]:
# Add a new column to the new DataFrame with the predicted clusters
cluster_df["Cluster Predictions"] = predictions

# Display sample data
cluster_df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Cluster Predictions
coin_id,,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761,0
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023,0
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954,3
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193,3
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384,0
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195,0
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186,0
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756,0
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408,0


In [15]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
cluster_df.hvplot.scatter(x="price_change_percentage_24h", y="price_change_percentage_7d", 
                          c="Cluster Predictions", cmap="colorblind_r", 
                          hover_cols=["Cluster Predictions", "coin_id"], title="Visualized KMeans Clusters")

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,Cluster Predictions,coin_id)

You might notice that the two points that are the sole inhabitants of their clusters, `ethlend` and `celsius-degree-token` are the outliers of their dataset even when scaled. Otherwise, the two groups, upon a casual glance at the chart, seem to be defined by "High (>5%)" and "Low (<5%)" weekly price changes.

---

### Optimize Clusters with Principal Component Analysis.

In [16]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

# Use the PCA model with `fit_transform` to reduce the data's dimensions to three principal components.
pca_data = pca.fit_transform(scaled_df)

# View the first five rows of the DataFrame. 
pca_data[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [17]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

[0.3719856  0.34700813 0.17603793]
0.8950316570309841


These 3 principal components seem to individually explain roughly 37.2%, 34.7%, and 17.6% of the variance of the dataset. Cumulatively, they explain 89.5% of the variance of the dataset.

In [18]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(pca_data, columns=["PCA_"+str(i+1) for i in range(len(pca.components_))])

# Copy the crypto names from the original data
pca_df = pca_df.set_index(df_market_data.index)

# Display sample data
pca_df.head()

,PCA_1,PCA_2,PCA_3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


In [19]:
# A quick component chart
weight_df = pd.DataFrame(pca.components_, columns=df_market_data.columns, 
                         index=["PCA_"+str(i+1) for i in range(len(pca.components_))])
weight_df

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
PCA_1,-0.416728,-0.102432,-0.009162,0.191523,0.320365,0.594468,0.568379
PCA_2,0.358162,0.227432,0.540415,0.562182,0.434040,0.027735,-0.150789
PCA_3,-0.218795,0.787670,0.349534,-0.181744,-0.361377,0.039733,0.210541


It seems that with this current model, PCA_1 scales in proportion to higher long-term changes (>200-day periods), PCA_2 scales more with mid-term changes(30 to 60-day periods), and PCA_3 scales with weekly changes.

For example, `bitcoin` has a high PCA_2 value and and negative PCA_1 value. This suggests that it did well in the scope of a monthly to bimonthly period, but performed less ideally in the long run (all relative to its competitors). This corresponds with what can be seen on the scaled data plot.

---

### Find the Best Value for k Using the PCA Data

In [20]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [21]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `pca_df`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, n_init="auto")
    model.fit(pca_df)
    inertia.append(model.inertia_)

C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known

In [22]:
# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame({"k":k, "inertia":inertia})
pca_elbow_df

,k,inertia
0,1,256.874086
1,2,168.811896
2,3,93.774626
3,4,49.665497
4,5,37.975977
5,6,32.182530
6,7,21.182776
7,8,17.566462
8,9,17.924353
9,10,11.430730


In [23]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_df.hvplot.line(x="k", y="inertia", xticks=k, title="PCA Clustering Elbow Plot")

:Curve   [k]   (inertia)

If we're using the same "elbow heuristic", the best value for `k` when using the PCA data still seems to be **4**, much like when we did regular KMeans clustering.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [24]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

# Fit the K-Means model using the PCA data
model.fit(pca_df)

# Predict the clusters to group the cryptocurrencies using the PCA data
predictions = model.predict(pca_df)

C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\user\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [25]:
# Print the resulting array of cluster values.
predictions

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0])

In [26]:
# Create a copy of the DataFrame with the PCA data
pca_k_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_k_df["Cluster Predictions"] = predictions

# Display sample data
pca_k_df.head()

,PCA_1,PCA_2,PCA_3,Cluster Predictions
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,3
ethereum,-0.458261,0.458466,0.952877,3
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,3


In [27]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_k_df.hvplot.scatter(x="PCA_1", y="PCA_2", 
                          c="Cluster Predictions", cmap="colorblind_r", 
                          hover_cols=["Cluster Predictions", "coin_id"], title="Visualized PCA KMeans Clusters")

:Scatter   [PCA_1]   (PCA_2,Cluster Predictions,coin_id)

### Comparing the PCA and non-PCA Results

In [28]:
# Composite plot to contrast the Elbow curves
no_pca_elbow = elbow_df.hvplot.line(x="k", y="inertia", xticks=k, 
                                    title="Cryptocurrency Clustering Elbow Plot", label="Without PCA")
pca_elbow = pca_elbow_df.hvplot.line(x="k", y="inertia", xticks=k, 
                                     title="PCA Clustering Elbow Plot", c="red", label="With PCA")

plt = no_pca_elbow*pca_elbow
plt

:Overlay
   .Curve.Without_PCA :Curve   [k]   (inertia)
   .Curve.With_PCA    :Curve   [k]   (inertia)

The difference between using PCA and not using PCA for this dataset is less pronounced as there are only 7 feature columns to begin with. However, it's notable that the PCA data has lower inertia than the non-PCA data for all levels of `k` and that the clusters themselves are slightly easier to distinguish with PCA dimensionality reduction done.

In [29]:
# Composite plot to contrast the clusters
no_pca_plot = cluster_df.hvplot.scatter(x="price_change_percentage_24h", y="price_change_percentage_7d", 
                          c="Cluster Predictions", cmap="colorblind_r", 
                          hover_cols=["Cluster Predictions", "coin_id"], 
                                        title="Visualized KMeans Clusters", label="Without PCA")
pca_plot = pca_k_df.hvplot.scatter(x="PCA_1", y="PCA_2", 
                          c="Cluster Predictions", cmap="colorblind_r", 
                          hover_cols=["Cluster Predictions", "coin_id"], 
                                   title="Visualized PCA KMeans Clusters", label="With PCA")
plt = no_pca_plot+pca_plot
plt

:Layout
   .Scatter.Without_PCA :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,Cluster Predictions,coin_id)
   .Scatter.With_PCA    :Scatter   [PCA_1]   (PCA_2,Cluster Predictions,coin_id)

If you compare the PCA clusters with the non-PCA clusters, you might notice that Clusters 1 and 2, our outliers, are significantly more distinct on the PCA cluster scatter plot than with the non-PCA version of the visualization. 

The cluster with `celsius-degree-token`, while bundled with the majority in the non-PCA cluster, now stands far away on its own in a different corner of the PCA plot.